In [1]:
# TRANSFER LEARNING FINE TUNING -  MASS CALC BENIGN MALIGN NORMAL CLASSIFIER USING MAMMOGRAM PATCHES IN CBIS-DDSM
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.layers import ReLU
import time

vid='v005'

In [2]:
#!pip install -U efficientnet

In [3]:
# Create CNN

IMG_SIZE=331

#from tensorflow.keras.applications.efficientnetb7 import EfficientNetB7
from efficientnet.tfkeras import EfficientNetB7
base_model=EfficientNetB7(weights='imagenet',include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable=False

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512)(x)
x=ReLU()(x)
x=Dropout(0.5)(x)
x=Dense(512)(x) 
x=ReLU()(x)
preds=Dense(5,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=preds)

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 331, 331, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 166, 166, 64) 1728        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 166, 166, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 166, 166, 64) 0           stem_bn[0][0]                    
______________________________________________________________________________________________

block6e_se_excite (Multiply)    (None, 11, 11, 2304) 0           block6e_activation[0][0]         
                                                                 block6e_se_expand[0][0]          
__________________________________________________________________________________________________
block6e_project_conv (Conv2D)   (None, 11, 11, 384)  884736      block6e_se_excite[0][0]          
__________________________________________________________________________________________________
block6e_project_bn (BatchNormal (None, 11, 11, 384)  1536        block6e_project_conv[0][0]       
__________________________________________________________________________________________________
block6e_drop (FixedDropout)     (None, 11, 11, 384)  0           block6e_project_bn[0][0]         
__________________________________________________________________________________________________
block6e_add (Add)               (None, 11, 11, 384)  0           block6e_drop[0][0]               
          

In [5]:
len(model.layers)

813

In [6]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy']) 

In [7]:
# Image preprocessing and data augmentation

batch_size=2

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         fill_mode="wrap",
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 90,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/train', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)
valid_generator = valid_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/valid', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)


Found 2170 images belonging to 5 classes.
Found 545 images belonging to 5 classes.


In [8]:
#Callbacks
#checkpoint_path="resnet50_train4/cp.ckpt"
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [9]:
#Training stage 1
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "mas_cal_ben_mal_class_nasnet50_weights_"+vid+".h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=3,
                   verbose=1,
                   callbacks = callbacks)  

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
1085/1085 [==============================] - 262s 206ms/step - loss: 1.6416 - accuracy: 0.3124 - val_loss: 1.4595 - val_accuracy: 0.3346


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/3
1085/1085 [==============================] - 215s 198ms/step - loss: 1.4592 - accuracy: 0.3774 - val_loss: 1.5308 - val_accuracy: 0.3309
Epoch 3/3
1085/1085 [==============================] - 215s 198ms/step - loss: 1.3485 - accuracy: 0.4258 - val_loss: 1.3886 - val_accuracy: 0.4062


In [10]:
#Training stage 2
for layer in model.layers[:545]:
    layer.trainable=False
for layer in model.layers[545:]:  #last ~33% layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=10,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/10
1085/1085 [==============================] - 379s 312ms/step - loss: 1.3810 - accuracy: 0.4244 - val_loss: 1.0825 - val_accuracy: 0.6048
Epoch 2/10
1085/1085 [==============================] - 338s 311ms/step - loss: 1.1957 - accuracy: 0.4793 - val_loss: 0.8954 - val_accuracy: 0.5864
Epoch 3/10
1085/1085 [==============================] - 333s 307ms/step - loss: 1.1344 - accuracy: 0.5055 - val_loss: 0.8907 - val_accuracy: 0.6471
Epoch 4/10
1085/1085 [==============================] - 333s 307ms/step - loss: 1.1329 - accuracy: 0.5143 - val_loss: 0.8623 - val_accuracy: 0.6618
Epoch 5/10
1085/1085 [==============================] - 333s 307ms/step - loss: 1.0378 - accuracy: 0.5576 - val_loss: 0.8119 - val_accuracy: 0.7059
Epoch 6/10
1085/1085 [==============================] - 332s 306ms/step - loss: 0.9735 - accuracy: 0.5816 - val_loss: 0.7984 - val_accuracy: 0.7426
Epoch 7/10
1085/1085 [==============================] - 334s 308ms/step - loss: 0.9187 - accuracy: 0.6240 - val_

In [11]:
#Training stage 3
#time.sleep(360)
for layer in model.layers:  #All layers are trainable
    layer.trainable=True
    
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = get_callbacks(name_weights = name_weights, patience_lr=5)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=150,
                   verbose=1,
                   callbacks = callbacks)

Epoch 1/150
1085/1085 [==============================] - 686s 587ms/step - loss: 1.0682 - accuracy: 0.5747 - val_loss: 0.7887 - val_accuracy: 0.6930
Epoch 2/150
1085/1085 [==============================] - 641s 591ms/step - loss: 0.9154 - accuracy: 0.6290 - val_loss: 0.8431 - val_accuracy: 0.6949
Epoch 3/150
1085/1085 [==============================] - 641s 591ms/step - loss: 0.8285 - accuracy: 0.6502 - val_loss: 0.7786 - val_accuracy: 0.7188
Epoch 4/150
1085/1085 [==============================] - 641s 590ms/step - loss: 0.8399 - accuracy: 0.6539 - val_loss: 0.8368 - val_accuracy: 0.7059
Epoch 5/150
1085/1085 [==============================] - 640s 590ms/step - loss: 0.7863 - accuracy: 0.6696 - val_loss: 0.8614 - val_accuracy: 0.7077
Epoch 6/150
1085/1085 [==============================] - 639s 589ms/step - loss: 0.7490 - accuracy: 0.6783 - val_loss: 0.7566 - val_accuracy: 0.7224
Epoch 7/150
1085/1085 [==============================] - 641s 590ms/step - loss: 0.7361 - accuracy: 0.6834

In [12]:
model.metrics_names

['loss', 'accuracy']

In [13]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('CBIS_DDSM_PATCHES_01/valid', 
                                                 target_size=(IMG_SIZE,IMG_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef


Y_pred = model.predict_generator(valid_generator)
y_pred = np.argmax(Y_pred, axis=1)
ras=roc_auc_score(valid_generator.classes, Y_pred,multi_class='ovr')


print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('ROC AUC score:', ras)



Found 545 images belonging to 5 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[ 65  39   1   3   1]
 [  5  85   2  16   1]
 [  0   2  48  56   3]
 [  2   7  14  86   0]
 [  0   0   0   3 106]]
Accuracy: 0.7155963302752294
ROC AUC score: 0.9407373116741015


In [20]:
y_pred

array([2, 2, 0, 4, 0, 1, 0, 0, 0, 3, 0, 1, 3, 1, 2, 0, 3, 3, 4, 3, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 2, 0, 3, 3, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 2, 2, 1, 1, 3, 0, 0, 1, 1, 1,
       1, 2, 3, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 4, 0, 1, 1, 3, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 4, 3, 1, 3, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 3, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 4, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4,
       1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 0, 1, 2,
       3, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 4, 3, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 0, 3, 1, 3, 3, 0, 1, 1, 1, 1, 1, 1, 0, 3, 1, 1, 1, 1, 4, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 0, 0, 1, 1, 3, 3, 2, 2, 3, 2,
       3, 3, 3, 2, 2, 3, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2,
       3, 3, 3, 3, 2, 3, 2, 2, 2, 3, 2, 3, 3, 3, 2,

In [14]:
train_generator.filenames

['calc_ben\\calc_ben_0001.png',
 'calc_ben\\calc_ben_0002.png',
 'calc_ben\\calc_ben_0003.png',
 'calc_ben\\calc_ben_0004.png',
 'calc_ben\\calc_ben_0006.png',
 'calc_ben\\calc_ben_0007.png',
 'calc_ben\\calc_ben_0008.png',
 'calc_ben\\calc_ben_0009.png',
 'calc_ben\\calc_ben_0017.png',
 'calc_ben\\calc_ben_0019.png',
 'calc_ben\\calc_ben_0020.png',
 'calc_ben\\calc_ben_0022.png',
 'calc_ben\\calc_ben_0023.png',
 'calc_ben\\calc_ben_0024.png',
 'calc_ben\\calc_ben_0025.png',
 'calc_ben\\calc_ben_0034.png',
 'calc_ben\\calc_ben_0035.png',
 'calc_ben\\calc_ben_0036.png',
 'calc_ben\\calc_ben_0038.png',
 'calc_ben\\calc_ben_0041.png',
 'calc_ben\\calc_ben_0049.png',
 'calc_ben\\calc_ben_0050.png',
 'calc_ben\\calc_ben_0051.png',
 'calc_ben\\calc_ben_0053.png',
 'calc_ben\\calc_ben_0054.png',
 'calc_ben\\calc_ben_0056.png',
 'calc_ben\\calc_ben_0057.png',
 'calc_ben\\calc_ben_0065.png',
 'calc_ben\\calc_ben_0067.png',
 'calc_ben\\calc_ben_0069.png',
 'calc_ben\\calc_ben_0071.png',
 'calc_b

In [15]:
valid_generator.filenames

['calc_ben\\calc_ben_0005.png',
 'calc_ben\\calc_ben_0018.png',
 'calc_ben\\calc_ben_0021.png',
 'calc_ben\\calc_ben_0033.png',
 'calc_ben\\calc_ben_0037.png',
 'calc_ben\\calc_ben_0039.png',
 'calc_ben\\calc_ben_0040.png',
 'calc_ben\\calc_ben_0052.png',
 'calc_ben\\calc_ben_0055.png',
 'calc_ben\\calc_ben_0066.png',
 'calc_ben\\calc_ben_0068.png',
 'calc_ben\\calc_ben_0070.png',
 'calc_ben\\calc_ben_0072.png',
 'calc_ben\\calc_ben_0081.png',
 'calc_ben\\calc_ben_0082.png',
 'calc_ben\\calc_ben_0089.png',
 'calc_ben\\calc_ben_0098.png',
 'calc_ben\\calc_ben_0102.png',
 'calc_ben\\calc_ben_0114.png',
 'calc_ben\\calc_ben_0115.png',
 'calc_ben\\calc_ben_0129.png',
 'calc_ben\\calc_ben_0137.png',
 'calc_ben\\calc_ben_0146.png',
 'calc_ben\\calc_ben_0147.png',
 'calc_ben\\calc_ben_0149.png',
 'calc_ben\\calc_ben_0150.png',
 'calc_ben\\calc_ben_0152.png',
 'calc_ben\\calc_ben_0166.png',
 'calc_ben\\calc_ben_0167.png',
 'calc_ben\\calc_ben_0184.png',
 'calc_ben\\calc_ben_0193.png',
 'calc_b

In [16]:
#Print estimation result
print(Y_pred)
print(valid_generator.classes)

[[9.9287355e-01 6.2986985e-03 4.4396022e-04 3.8321820e-04 5.2275612e-07]
 [9.3247139e-01 5.3265546e-02 6.5986714e-03 7.6315161e-03 3.2915916e-05]
 [1.3305926e-01 8.4207940e-01 3.1970690e-03 2.1644348e-02 1.9900448e-05]
 ...
 [7.9837707e-07 2.8864540e-06 1.6014551e-06 5.0267175e-07 9.9999416e-01]
 [2.2712208e-03 3.1839139e-03 1.1026118e-02 8.9882808e-03 9.7453046e-01]
 [1.7039076e-06 4.2742581e-06 2.7225381e-06 1.0166322e-06 9.9999022e-01]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 